# Dask Delayed

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Dask Delayed"
* https://docs.dask.org/en/latest/delayed.html
* Jesse C. Daniel. Data Science with Python and Dask.
* https://saturncloud.io/blog/a-data-scientist-s-guide-to-lazy-evaluation-with-dask/
* https://www.coiled.io/blog/how-to-learn-dask-in-2021

## Задачи для совместного разбора

![](https://i.imgur.com/AwiN8y6.png)
![](https://i.imgur.com/ceY6guU.png)

1\. Напишите 2 функции, имитирующие CPU-bound задачу и IO-bound задачу:

`cpu_task()`: генерирует 100 тыс. случайных чисел и возвращает их сумму (без использования `numpy`)

`io_task()`: "спит" 0.1 сек, затем генерирует случайное число и возвращает его

Замерьте время выполнения 100 последовательных вызовов каждой из этих функций. Распараллелив вычисления при помощи `dask.delayed`, сократите время выполнения. Исследуйте, как зависит время вычислений от выбранного планировщика `scheduler`.

In [1]:
import numpy as np
import dask.delayed as delayed
import dask
import dask.dataframe as dd
import dask.bag as db
from bs4 import BeautifulSoup
from time import sleep
import random
import pandas as pd

In [3]:
def cpu_task():
    numbers = [random.randint(0, 1000) for _ in range(100_000)]
    return sum(numbers)

def io_task():
    sleep(0.1)
    return random.randint(0, 1000)

In [4]:
%%time
r = [cpu_task() for _ in range(100)]

CPU times: total: 22.1 s
Wall time: 22.1 s


In [5]:
%%time
r = [io_task() for _ in range(100)]

CPU times: total: 0 ns
Wall time: 10.8 s


In [6]:
cpu_task_delayed = dask.delayed(cpu_task)

In [7]:
%%time
r = [cpu_task_delayed() for _ in range(100)]
r = dask.compute(r, scheduler="threading")

CPU times: total: 23.6 s
Wall time: 23.6 s


In [8]:
%%time
r = [cpu_task_delayed() for _ in range(100)]
r = dask.compute(r, scheduler="multiprocessing")

CPU times: total: 203 ms
Wall time: 9.12 s


In [9]:
io_task_delayed = dask.delayed(io_task)

In [10]:
%%time
r = [io_task_delayed() for _ in range(100)]
r = dask.compute(r, scheduler="threading")

CPU times: total: 15.6 ms
Wall time: 1.44 s


In [11]:
%%time
r = [io_task_delayed() for _ in range(100)]
r = dask.compute(r, scheduler="multiprocessing")

CPU times: total: 78.1 ms
Wall time: 3.12 s


## Лабораторная работа 14

1\. Напишите функцию, которая считывает файл формата xml из каталога `reviewers_full` и по данным этого файла формирует список словарей, содержащих следующие ключи: `id`, `username`, `name`, `sex`, `country`, `mail`, `registered`, `birthdate`, `name_prefix`, `country_code`. Часть из этих значений в исходном файле хранится в виде тэгов, часть - в виде атрибутов тэгов. Для конкретного человека какие-то из этих ключей могут отсутствовать. 



In [ ]:
# ./14_delayed_data/reviewers_full/reviewers_full_{path}.xml

In [24]:
def xml_to_dict(path):
    reviewers_full = []
    fp = open(path, "r").read()
    xml = BeautifulSoup(fp,'xml')
    
    for user in xml.find_all('user'):
        user_dict = {}
        for key in ['id', 'username', 'name', 'sex', 'country', 'mail', 'registered', 'birthdate']:
            if user.find(key):
                user_dict[key] = user.find(key).text
        if user.attrs:
            user_dict['name_prefix'] = user.attrs['prefix']
        if user.find('country'):
            code = user.find('country').attrs
            if code:
                user_dict['country_code'] = code['code']
            
        reviewers_full.append(user_dict)
    return reviewers_full

In [25]:
xml_to_dict('./14_delayed_data/reviewers_full/reviewers_full_0.xml')[:2]

[{'id': '88005',
  'username': 'jacqueline00',
  'name': 'Michele Lewis',
  'mail': 'morenocharlotte@yahoo.com'},
 {'id': '68591',
  'username': 'daniellegomez',
  'sex': 'F',
  'country': 'Germany',
  'birthdate': '2005-03-06',
  'name_prefix': 'Dr.',
  'country_code': 'DE'}]

2\. Измерьте время выполнения функции из задания 1 на всех файлах из каталога `reviewers_full`. Ускорьте время выполнения, используя `dask.delayed`.

### Выполнение без ускорения:

In [64]:
%%time
for i in range(20):
    xml_to_dict(f'./14_delayed_data/reviewers_full/reviewers_full_{i}.xml')

CPU times: total: 17min 53s
Wall time: 18min 20s


### Выполнение с dask.delayed:

In [12]:
xml_to_dict_delayed = dask.delayed(xml_to_dict)

In [68]:
%%time
r = [xml_to_dict_delayed(f'./14_delayed_data/reviewers_full/reviewers_full_{i}.xml') for i in range(20)]
r = dask.compute(r, scheduler="multiprocessing")

CPU times: total: 2.09 s
Wall time: 5min 54s


In [26]:
%%time
r = [xml_to_dict_delayed(f'./14_delayed_data/reviewers_full/reviewers_full_{i}.xml') for i in range(20)]
r = dask.compute(r, scheduler="threading")

CPU times: total: 20min 44s
Wall time: 19min 56s


3\. Задекорируйте функцию из задания 1 при помощи `dask.delayed` и создайте список `reviewers`, состоящий из 5 объектов `delayed` (по одному объекту на файл). Из списка объектов `delayed`, создайте `dask.bag` при помощи метода `db.from_delayed`. Добавьте ключ `birth_year`, в котором хранится год рождения человека. Оставьте в выборке только тех людей, которые __наверняка__ моложе 1980 года. Преобразуйте поле `id` к целому типу.

In [13]:
from datetime import datetime

In [14]:
reviewers = [xml_to_dict_delayed(f'./14_delayed_data/reviewers_full/reviewers_full_{i}.xml') for i in range(5)]

In [15]:
len(reviewers)

5

### Преобразование поля "id" к целому типу. Создание дополнительного столбца birth_year

In [16]:
def upgrate_review(user):
    user["id"] = int(user["id"])
    
    if user.get("birthdate"): 
        date = datetime.strptime(user["birthdate"], '%Y-%m-%d')
        user["birth_year"] = date.year
        
    return user

In [17]:
db_reviewers = db.from_delayed(reviewers).map(upgrate_review)

In [18]:
db_reviewers.take(4)

({'id': 88005,
  'username': 'jacqueline00',
  'name': 'Michele Lewis',
  'mail': 'morenocharlotte@yahoo.com'},
 {'id': 68591,
  'username': 'daniellegomez',
  'sex': 'F',
  'country': 'Germany',
  'birthdate': '2005-03-06',
  'name_prefix': 'Dr.',
  'country_code': 'DE',
  'birth_year': 2005},
 {'id': 81003,
  'username': 'alucero',
  'name': 'Tammy Patton',
  'mail': 'larsenrobert@gmail.com'},
 {'id': 61509,
  'username': 'mcleanjacqueline',
  'name': 'Jeremy Schmidt',
  'sex': 'M',
  'registered': '2017-09-21',
  'name_prefix': 'Dr.'})

### Оставим в выборке только тех людей, которые наверняка моложе 1980 года.

In [19]:
def filter_1980(user):
    return user.get("birth_year") is not None and user["birth_year"] > 1980

In [20]:
db_rev_1980 = db_reviewers.filter(filter_1980)

In [21]:
db_rev_1980.take(4)

({'id': 68591,
  'username': 'daniellegomez',
  'sex': 'F',
  'country': 'Germany',
  'birthdate': '2005-03-06',
  'name_prefix': 'Dr.',
  'country_code': 'DE',
  'birth_year': 2005},
 {'id': 43614,
  'username': 'jeffreynelson',
  'sex': 'F',
  'country': 'Aruba',
  'mail': 'laura77@hotmail.com',
  'registered': '2019-11-19',
  'birthdate': '1994-10-10',
  'country_code': 'AW',
  'birth_year': 1994},
 {'id': 25362,
  'username': 'davidmaxwell',
  'mail': 'steven53@gmail.com',
  'registered': '2005-11-28',
  'birthdate': '1996-09-07',
  'birth_year': 1996},
 {'id': 1509,
  'username': 'thomas07',
  'sex': 'F',
  'country': 'Tajikistan',
  'registered': '2016-09-07',
  'birthdate': '1997-03-29',
  'name_prefix': 'Dr.',
  'country_code': 'TJ',
  'birth_year': 1997})